<a href="https://colab.research.google.com/github/mehdihosseinimoghadam/Pytorch-Tutorial/blob/main/Simple_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torch import nn
import torchvision.transforms as transformers
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch import optim

In [ ]:
from torch.autograd import backward
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


num_classes = 10
num_epoch = 3
hidden_size = 256
num_layers = 2
input_size = 28
sequence_length = 28
learning_rate = .001
batch_size = 64




class RNN(nn.module):
  def __init__(self, input_size, num_layers, hidden_size, num_classes): 
    super(RNN, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.rnn = nn.RNN(input_size,
                      hidden_size,
                      num_layers,
                      batch_first=True)
    
    self.fc = nn.Linear(hidden_size*sequence_length, num_classes)


  def forward(self, x):
    h0 = torch.zeros(self.num_layers, 
                     x.size[0], 
                     hidden_size).to(device)  

    out, _ = self.rnn(x, h0)
    out = out.reshape(out.shape[0], -1)

    out = self.fc(out) 
    return out




train_data = datasets.MNIST("data/",
                            train = True,
                            transform = transformers.ToTensor(),
                            download = True)                    


test_data = datasets.MNIST("data/",
                            train = False,
                            transform = transformers.ToTensor(),
                            download = True)

treain_loader = DataLoader(train_data,
                          batch_size = batch_size,
                          shuffle = True)


test_loader = DataLoader(test_data,
                          batch_size = batch_size,
                          shuffle = True
                         

model = RNN(input_size,
            num_layers,
            hidden_size,
            num_classes)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=learning_rate)


for epoch in range(num_epochs):
  for batch_idx , (data, targets) in enumerate(train_loader):
    data = data.to(device)
    targets = targets.to(device)

    scores = model(data)

    loss = criterion(targets, scores)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()



def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0

    # Set model to eval
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device).squeeze(1)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    # Toggle model back to train
    model.train()
    return num_correct / num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:2f}")
print(f"Accuracy on test set: {check_accuracy(test_loader, model)*100:.2f}")